In [48]:
import numpy as np
import awkward as ak
np.random.seed(1337)  # for reproducibility

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import utils as pic

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
lr_init     = 1.e-3    # Initial learning rate  
batch_size  = 64       # Training batch size
train_size  = 1024     # Training size
valid_size  = 1024     # Validation size
test_size   = 1024     # Test size
epochs      = 20       # Number of epochs
doGPU       = False    # Use GPU

In [3]:
# Set range of training set
train_start, train_stop = 0, train_size
assert train_stop > train_start
assert (len(pic.decays)*train_size) % batch_size == 0
X_train, y_train = pic.load_data(train_start,train_stop)

# Set range of validation set
valid_start, valid_stop = 160000, 160000+valid_size
assert valid_stop  >  valid_start
assert valid_start >= train_stop
X_valid, y_valid = pic.load_data(valid_start,valid_stop)

# Set range of test set
test_start, test_stop = 204800, 204800+test_size
assert test_stop  >  test_start
assert test_start >= valid_stop
X_test, y_test = pic.load_data(test_start,test_stop)

samples_requested = len(pic.decays) * (train_size + valid_size + test_size)
samples_available = len(y_train) + len(y_valid) + len(y_test)
assert samples_requested == samples_available

In [5]:
X_train = pic.remove_empty_pixels(X_train)
X_e_train,X_t_train,maxframes = pic.timeordered(X_train,cumulative=False)
X_e_train = np.where(np.isnan(X_e_train),0,X_e_train)
X_e_train = np.where(np.isnan(X_t_train)[:,:,None],-69420,X_e_train.reshape(2*train_size,-1,32*32)).reshape(X_e_train.shape)
y_train = to_categorical(y_train)

In [14]:
X_e_train.shape

(2048, 78, 32, 32, 1)

In [6]:
X_test = pic.remove_empty_pixels(X_test)
X_e_test,X_t_test,maxframes = pic.timeordered(X_test,cumulative=False)
X_e_test = np.where(np.isnan(X_e_test),0,X_e_test)
X_e_test = np.where(np.isnan(X_t_test)[:,:,None],-69420,X_e_test.reshape(2*test_size,-1,32*32)).reshape(X_e_test.shape)
y_test = to_categorical(y_test)

In [25]:
model = keras.Sequential()

model.add(layers.Masking(mask_value=-69420, input_shape=(None, 32, 32, 1)))
# model.add(layers.ConvLSTM2D(3,3,activation='relu',recurrent_activation='relu',padding='same'))
# model.add(layers.TimeDistributed(layers.Flatten()))
# model.add(layers.Dense(2,activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9 (Masking)          (None, None, 32, 32, 1)   0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [23]:
history = model.fit(
    X_e_train, y_train,
    validation_split=0.2,
    epochs=3,
    batch_size=50,
    shuffle=True,
    verbose=1
)

Epoch 1/3
33/33 [==============================] - 140s 4s/step - loss: 0.6914 - accuracy: 0.6252 - val_loss: 0.7079 - val_accuracy: 0.0000e+00
Epoch 2/3
33/33 [==============================] - 137s 4s/step - loss: 0.6841 - accuracy: 0.6252 - val_loss: 0.9077 - val_accuracy: 0.0000e+00
Epoch 3/3
22/33 [===================>..........] - ETA: 41s - loss: 0.6771 - accuracy: 0.6291

KeyboardInterrupt: 

In [1]:
model

NameError: name 'model' is not defined